In [5]:
#Imports Functions for Building Hamiltonains
%run Utils.ipynb

In [89]:
m = 1

x_res = 10
x_min, x_max = 0.0001, 0.01

n_max = 12
n_res = (n_max-4)//2 + 1

x_val, qubits = np.meshgrid(1 / (np.linspace(x_min, x_max, x_res) ** 2), np.linspace(4, n_max ,n_res))
f0 = np.zeros([n_res, x_res], dtype=float)
f1 = np.zeros([n_res, x_res], dtype=float)
f2 = np.zeros([n_res, x_res], dtype=float)

for i in range(n_res):
    for j in range(x_res):
        f0[i,j], f1[i,j], f2[i,j] = findDimensionlessEigen(x_val[i,j], qubits[i,j], m)
        print(f'{((i * x_res) + j + 1) / (x_res * n_res) * 100 :3.2f}%   Qubit Count: {4 + i * 2}', end='\r', flush = True)
print('\nDone!')

100.00%   Qubit Count: 12
Done!


# Ground State Convergence via Extrapolation

We extrapolate the axes of $\frac{1}{\sqrt(x)}$ and $\frac{1}{N}$ to $0$ in order to find the ground state at $x \rightarrow \infty$ and $N \rightarrow \infty$

In [91]:
x_ext = np.linspace(0, x_max , 10)
n_ext = np.linspace(0, .25 , 8)

extrap_x_f0, extrap_y_f0, extrap_poly_f0 = gen_extrapolated_data(fx(x_val), fy(qubits), f0, n_ext, 'y')
extrap_x_f0, extrap_y_f0, extrap_poly_f0 = gen_extrapolated_data(extrap_x_f0, extrap_y_f0, extrap_poly_f0[1], x_ext, 'x')

extrap_f0 = extrap_poly_f0[1]

In [99]:
extrapolated, actual = extrap_f0[0,0], -1/np.pi
error = abs((extrapolated - actual) / actual) * 100

fig = go.Figure(data=[go.Surface(x=fx(x_val), y=fy(qubits), z=f0, opacity=0.8, showscale=False, colorscale='Viridis'), 
                      go.Surface(x=extrap_x_f0, y=extrap_y_f0, z=extrap_f0, opacity=0.4, showscale=False, colorscale='Viridis'),
                      go.Surface(x=extrap_x_f0, y=extrap_y_f0, z=np.full([len(extrap_x_f0), len(extrap_y_f0.T)], -1/np.pi), opacity=0.3, showscale=False, colorscale='Thermal'),
                      ])

fig.add_scatter3d(x = [0], y = [0], z = [extrapolated])

fig.update_layout(title='Extrapolated w0 Convergence',
                   autosize=False,
                  width=1000, height=700,
                 scene = dict(
                    xaxis_title='1/sqrt(x)',
                    yaxis_title='1/N',
                    zaxis_title='Energy'))
fig.show()

print(f"{extrapolated:1.6f} - Extrapolated Convergence")
print(f"{actual:1.6f} - Actual Convergence")
print(f"\nError {error: 2.3f}%")

-0.318052 - Extrapolated Convergence
-0.318310 - Actual Convergence

Error  0.081%


# Check Eigenvalues Against Actual Values

This test compares the eigenvalues a Schwinger Model from a different method with our own Schwinger Model. The Error is the sum total of the difference between each eigenvalue.

In [86]:
%run Utils.ipynb
import csv
def open_test_spectra(file_name):
    with open(f'schwinger_test_spectra/{file_name}.csv', newline='') as csvfile:
        data = np.array([data[0] for data in list(csv.reader(csvfile))]).astype('float64')
    return data

for N in range(4, 10, 2):
    for x, m in zip([1,1,10], [0.1, 1, 1]):
        print(f"N = {N}, x = {x}, m/g = {m}")
        calc_val = dimensionless_eigv(N, x, m)
        ref_val = open_test_spectra(f'schwinger_spectrum_N{N}_x{x}_mg{m}')
        error = sum(abs(calc_val - ref_val))
        print(f"Error: {error: 1.2e}\n")

N = 4, x = 1, m/g = 0.1
Error:  1.22e-14

N = 4, x = 1, m/g = 1
Error:  1.15e-14

N = 4, x = 10, m/g = 1
Error:  7.74e-14

N = 6, x = 1, m/g = 0.1
Error:  2.46e-13

N = 6, x = 1, m/g = 1
Error:  2.30e-13

N = 6, x = 10, m/g = 1
Error:  1.40e-12

N = 8, x = 1, m/g = 0.1
Error:  4.00e-12

N = 8, x = 1, m/g = 1
Error:  3.34e-12

N = 8, x = 10, m/g = 1
Error:  1.31e-11

